# The Relationship Between a Hotel's Geographical Proximity to the Center of the Country and Its Price

* Yarin Cohen, ID: 211361720
* Amit Shiber, ID: 322372582

## About Our Project

From time to time the issue of the periphery versus the center of the country comes up in the media. We decided to research more about the subject and check the hotel prices in the cities near Tel Aviv and in the distant cities. After crawling the data from the hotel website, we will use an additional function with an API to calculate distances between two locations. Is there a connection between the price of hotel charges and its proximity to the center of country?

### Information Sources and Data Acquisition Methods

* **Crawling Booking.com** - One of the largest online travel agencies. As of December 31, 2022, Booking.com offered lodging reservation services for approximately 2.7 million properties, including 400,000 hotels, motels, and resorts and 2.3 million homes, apartments in over 220 countries and in over 40 languages. It will help us getting data about the hotels in this project.

* **GeoDB Cities API** - Online cities database. It exposes city, region, and country data via both GraphQL and REST APIs. It will help us calculate the distance between two cities.

### Data Set Description

Each line in the data set represents a hotel.

Columns representation in the data set:
* Hotel name
* Hotel Address
* Hotel Description
* Price per night (on a fixed date, the cheapest deal)
* Score - general
* Score - stuff
* Score - facilities
* Score - convenience
* Score - value for money
* Score - location
* Score - clean
* Proximity to the center of the country (km)

### Machine Learning

* **Type of ML**: Regression

* We will start with easy regression models (one variable and low powers) and try to go through each pair of an explanatory variable and an explained variable.

* There is no rule that says how many variables make a regression heavy and sluggish. If the software starts to falter, we will stop and think whether adding the variables and holdings will contribute to the prediction or only to the complications of calculation, memory, etc. We are required to exercise discretion between predictability and complications and resources such as private time.

* If the learning results are not satisfactory, we will use
in classification and division into price levels.

## Imports

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import time
import random
from tqdm import tqdm
import pandas as pd
import scipy as sc
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import sklearn
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score
%matplotlib inline

## Step 1: Defining a Research Question

Is it possible to predict the price of a night in a certain hotel, based on its proximity to the center and the score given to it by surfers in the various categories?

## Step 2: Data Acquisition

### Data Acquisition by Crawling

First of all, we will check Booking.com's Robots.txt terms, to understand if there are any pages we can't crawl: https://booking.com/robots.txt

* We will start by searching manually on Booking's main page for a vacation in Israel, on 01-02/08/2023.

* The results page will be crawled first.
* Due to complexity of HTML elements, we will use the mobile version of Booking.
* <a href="https://www.booking.com/searchresults.he.html?ss=%D7%99%D7%A9%D7%A8%D7%90%D7%9C&ss=%D7%99%D7%A9%D7%A8%D7%90%D7%9C&group_adults=2&group_children=0&no_rooms=1&sb_travel_purpose=leisure&ssne=%D7%99%D7%A9%D7%A8%D7%90%D7%9C&ssne_untouched=%D7%99%D7%A9%D7%A8%D7%90%D7%9C&sb_changed_dates=1&label=gen173nr-1BCAEoggI46AdIM1gEaGqIAQGYAQ64AQfIAQzYAQHoAQGIAgGoAgO4AsPO_6IGwAIB0gIkN2EzYmVmMjgtNTkwYS00YjMyLWI5ZmUtMmZjMTQwOTdmM2I42AIF4AIB&sid=ae3ca57b743d1747c5f828a2fabc4587&aid=304142&lang=he&sb=1&src_elem=sb&src=searchresults&dest_id=103&dest_type=country&checkin=2024-02-01&checkout=2024-02-02&prefer_site_type=mdot" >This is</a> the first page will be crawled.

#### Auxiliary Functions

In [2]:
# Load soup object:

def loadSoupObject(url):

    headers = { "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148" }
    
    time.sleep(random.randint(1,5))
    r = requests.get(url, headers=headers).content
    
    return BeautifulSoup(r,"html.parser")

In [3]:
# Getting URLs of all the hotels in the page:

def getHotelsURL(soupObj):

    links = []
   
    for link in soupObj.findAll("a", {"data-testid" : "title"}):
        links.append(link.get("href"))

    return links

In [4]:
# Getting URL of the next results page:

def getNextPage(soupObj):
    return soupObj.find("a", {"title" : "Next page"}).get("href")

In [5]:
# Getting information from a hotel page:

def getHotelData(soupObj):

    dataOfHotel = []

    # Hotel name:
    name = soupObj.find("span",{"class" : "hp-header--title--text"})
    dataOfHotel.append(name.text if name else np.nan)

    # Hotel address:
    add = soupObj.find("span",{"class" : "js_hp_address_text_line"})
    dataOfHotel.append(add.text if add else np.nan)

    # Hotel description:
    desc = soupObj.find("div",{"class" : "page-section--content"})
    dataOfHotel.append(desc.text if desc else np.nan)
    
    # Price per night (on a fixed date, the cheapest deal):
    price = soupObj.find("div",{"class" : "prco-js-headline-price"})
    dataOfHotel.append(price.text if price else np.nan)

    # Score - general:
    score = soupObj.find("div",{"data-testid" : "review-score-component"})
    dataOfHotel.append(score.text if score else np.nan)

    # Score - value for money:
    value = soupObj.find("div",{"id" : ":R5m:-label"})
    dataOfHotel.append(value.text if value else np.nan)

    # Score - location:
    location = soupObj.find("div",{"id" : ":R4m:-label"})
    dataOfHotel.append(location.text if location else np.nan)

    # Score - clean:
    clean = soupObj.find("div",{"id" : ":R56:-label"})
    dataOfHotel.append(clean.text if clean else np.nan)

    return dataOfHotel


In [6]:
# The whole process of crawling all the hotels from results pages:

def getAllHotels(urlResults):

    currentPage = loadSoupObject(urlResults)

    resultsPages = []
    hotelsLinks = []

    # Collecting links of results pages:

    resultsPages.append(currentPage)

    for i in tqdm(range(33), desc="Collecting next-links..."):
        
        if (currentPage.find("a", {"title" : "Next page"})):
            nextPage = loadSoupObject(getNextPage(currentPage))
            resultsPages.append(nextPage)
            currentPage = nextPage

            if not (currentPage.find("a", {"title" : "Next page"})):
                break

    # Collecting links of hotels:

    for page in tqdm(resultsPages,desc="Collecting links from result pages..."):
        hotelsLinks.extend(getHotelsURL(page))

    # Crawling data from hotel pages

    hotelsData = []

    for link in tqdm(hotelsLinks, desc="Crawling hotels data..."):
        hotelsData.append(getHotelData(loadSoupObject(link)))

    return hotelsData

#### Main Function

First, we start by crawling the results pages when searching for "Israel", "Herzliya", etc. in the search bar found on the home page.
- Crawling all "next page" links found on the results pages.

- Crawling all the links of the hotel pages found in the results pages.
- Crawling the data from each hotel page.

Here we can see an array of 18 links, which were obtained as a result of searching these phrases on the homepage of Booking.com.

These links are not displayed in one place on Booking.com. Therefore, we had to copy and paste them, instead of crawling them as well.

In [7]:
searches = [
    # All of Israel:
    "https://www.booking.com/searchresults.en-us.html?ss=israel&ssne=Upper+Galilee&ssne_untouched=Upper+Galilee&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=103&dest_type=country&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=57b8636d07030080&ac_meta=GhA1N2I4NjM2ZDA3MDMwMDgwIAAoATICZW46BWlzcmFlQABKAFAA&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Herzelia:
    "https://www.booking.com/searchresults.en-us.html?ss=Herzelia+%2C+Center+District+Israel%2C+Israel&ssne=Israel&ssne_untouched=Israel&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=-780136&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=347f65e8986607e2&ac_meta=GhAzNDdmNjVlODk4NjYwN2UyIAAoATICZW46B2hlcnplbGlAAEoAUAA%3D&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Netanya:
    "https://www.booking.com/searchresults.en-us.html?ss=Netanya&ssne=Herzliya&ssne_untouched=Herzliya&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Mizpe Ramon:
    "https://www.booking.com/searchresults.en-us.html?ss=Mizpe+Ramon&ssne=Netanya&ssne_untouched=Netanya&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=900040703&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=54a2660fa2bc0084&ac_meta=GhA1NGEyNjYwZmEyYmMwMDg0IAAoATICZW46C01penBlIFJhbW9uQAFKDG1pdHpwZSByYW1vblCGAQ%3D%3D&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Yeruham:
    "https://www.booking.com/searchresults.en-us.html?ss=Yeruham&ssne=Mitzpe+Ramon&ssne_untouched=Mitzpe+Ramon&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Haifa:
    "https://www.booking.com/searchresults.en-us.html?ss=Haifa&ssne=Yero%E1%BA%96am&ssne_untouched=Yero%E1%BA%96am&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Nazareth:
    "https://www.booking.com/searchresults.en-us.html?ss=Nazareth%2C+North+District+Israel%2C+Israel&ssne=Haifa&ssne_untouched=Haifa&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=-780833&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b924663990bd0516&ac_meta=GhBiOTI0NjYzOTkwYmQwNTE2IAAoATICZW46CE5hemFyZXRoQABKAFAA&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Israel's north district:
    "https://www.booking.com/searchresults.en-us.html?ss=North+District+Israel%2C+Israel&ssne=Nazareth&ssne_untouched=Nazareth&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=3638&dest_type=region&ac_position=1&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=25686645b89502a2&ac_meta=GhAyNTY4NjY0NWI4OTUwMmEyIAEoATICZW46B2lzcmFlbCBAAEoAUAA%3D&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Israel's center district:
    "https://www.booking.com/searchresults.en-us.html?ss=Center+District+Israel%2C+Israel&ssne=North+District+Israel&ssne_untouched=North+District+Israel&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=3640&dest_type=region&ac_position=3&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=45f86657a1f70228&ac_meta=GhA0NWY4NjY1N2ExZjcwMjI4IAMoATICZW46BmlzcmFlbEAASgBQAA%3D%3D&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Israel's south district:
    "https://www.booking.com/searchresults.en-us.html?ss=South+District+Israel%2C+Israel&ssne=Center+District+Israel&ssne_untouched=Center+District+Israel&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=3642&dest_type=region&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=fc976674c81401ea&ac_meta=GhBmYzk3NjY3NGM4MTQwMWVhIAAoATICZW46FXNvdXRoIERpc3RyaWN0IElzcmFlbEAASgBQAA%3D%3D&checkin=2023-09-03&checkout=2023-09-04&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Tel Aviv:
    "https://www.booking.com/searchresults.en-us.html?ss=Tel+Aviv&ssne=Tel+Aviv&ssne_untouched=Tel+Aviv&efdco=1&label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=-781545&dest_type=city&checkin=2023-09-03&checkout=2023-09-04&ltfd=1%3A1%3A9-2023%3A&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure",
    # Jerusalem:
    "https://www.booking.com/searchresults.html?label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&checkin=2023-09-03&checkout=2023-09-04&dest_id=900000000&dest_type=city&group_adults=null&req_adults=null&no_rooms=null&group_children=null&req_children=null",
    # Dead Sea:
    "https://www.booking.com/searchresults.html?label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&checkin=2023-09-03&checkout=2023-09-04&dest_id=3230&dest_type=region&group_adults=null&req_adults=null&no_rooms=null&group_children=null&req_children=null",
    # Tiberias:
    "https://www.booking.com/searchresults.html?label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&checkin=2023-09-03&checkout=2023-09-04&dest_id=-781620&dest_type=city&group_adults=null&req_adults=null&no_rooms=null&group_children=null&req_children=null",
    # Golan Heights:
    "https://www.booking.com/searchresults.html?label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&checkin=2023-09-03&checkout=2023-09-04&dest_id=3644&dest_type=region&group_adults=null&req_adults=null&no_rooms=null&group_children=null&req_children=null",
    # Safed:
    "https://www.booking.com/searchresults.html?label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&checkin=2023-09-03&checkout=2023-09-04&dest_id=-781845&dest_type=city&group_adults=null&req_adults=null&no_rooms=null&group_children=null&req_children=null",
    # Zikhron Yaakov:
    "https://www.booking.com/searchresults.html?label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&checkin=2023-09-03&checkout=2023-09-04&dest_id=-781878&dest_type=city&group_adults=null&req_adults=null&no_rooms=null&group_children=null&req_children=null",
    # Upper Galile:
    "https://www.booking.com/searchresults.html?label=gen173nr-1FCAEoggI46AdIM1gEaGqIAQGYATG4AQfIAQzYAQHoAQH4AQKIAgGoAgO4AtDFqKMGwAIB0gIkMjI2NjFiN2ItODhhYS00ZmYwLWIyMDUtMzdlOTc5OWU5OGZi2AIF4AIB&sid=863fbc6e6306cb97b7ec9fa1361b67a2&aid=304142&checkin=2023-09-03&checkout=2023-09-04&dest_id=7336&dest_type=region&group_adults=null&req_adults=null&no_rooms=null&group_children=null&req_children=null"
]

fullData = []

In [8]:
for index, url in enumerate(searches):
    print("Cycle",index+1)
    fullData.extend(getAllHotels(url))
    print("")

Cycle 1


Crawling hotels data...: 100%|██████████| 1020/1020 [1:44:12<00:00,  6.13s/it]



Cycle 2


Crawling hotels data...: 100%|██████████| 591/591 [58:02<00:00,  5.89s/it]  



Cycle 3


...

Crawling hotels data...: 100%|██████████| 66/66 [06:01<00:00,  5.48s/it]



Cycle 17


Crawling hotels data...: 100%|██████████| 147/147 [13:03<00:00,  5.33s/it]



Cycle 18


Crawling hotels data...: 100%|██████████| 586/586 [51:59<00:00,  5.32s/it] 

In [9]:
# Creating a data set:

headers = ["hotelName", "hotelAddress", "hotelDescription", "pricePerNight","scoreGeneral","scoreValueForMoney","scoreLocation","scoreClean"]

df = pd.DataFrame(fullData, columns=headers)
df

,hotelName,hotelAddress,hotelDescription,pricePerNight,scoreGeneral,scoreValueForMoney,scoreLocation,scoreClean
0,Melio Hotel Eilat,"\nSderot Hativat HaNegev 20, Eilat\n","\nSet in Eilat, within 1.2 km of Kisuski Beach...",\n₪ 286\n,"7.9 Good · 1,075 reviews",8.4,7.8,8.4
1,Badolina Ein Gedi Glamping,"\nEin Gedi Street, Ein Gedi\n","\nDead Sea Camping is set in Ein Gedi, within ...",\n₪ 130\n,8.5 Very good · 814 reviews,7.7,9.2,8.3
2,Daniel Herzliya Hotel,"\n60 Ramat Yam Street, Herzelia \n","\nSet right on the beach, in the heart of Herz...",\n₪ 777\n,7.9 Good · 162 reviews,7.5,9.0,8.0
3,Merom Golan Resort,"\nKibbutz Merom Golan, Merom Golan\n",\nMerom Golan Resort offers quiet mountain acc...,\n₪ 983\n,"8.7 Fabulous · 1,400 reviews",7.9,9.4,8.9
4,Leonardo Club Hotel Dead Sea - All Inclusive,"\nEin Bokek (Dead sea), Ein Bokek\n",\nThis all-inclusive deluxe hotel is set a pri...,"\n₪ 1,308\n","7.7 Good · 6,414 reviews",7.4,8.7,7.5
...,...,...,...,...,...,...,...,...
6076,צימר נוף העמק,"\n28 נוף העמק, Nof Kinneret\n","\nSituated in Nof Kinneret, 4.7 km from Israel...","\n₪ 1,454\n",8.0 Very good · 1 review,10,10,10
6077,Supreme 4BR Villa With Garden and Sea View,"\n55 Jabotinski Street, Nahariyya\n",\nSituated in Nahariyya and only 400 m from Ha...,"\n₪ 4,087\n",NaN,NaN,NaN,NaN
6078,Blue lake luxury villa for families,"\nעין אל אסד, ‘Ein el Asad\n",\nLocated in ‘Ein el Asad and only 37 km from ...,"\n₪ 4,160\n",6.0 Pleasant · 1 review,NaN,NaN,NaN
6079,Paradise by the sea,"\nדוגית, Nahariyya\n","\nParadise by the sea is located in Nahariyya,...","\n₪ 2,880\n",NaN,NaN,NaN,NaN


In [12]:
# Export data set to csv:
df.to_csv("Data/data.csv", index=False)

In [3]:
# Load data set from csv (for personal use):
df = pd.read_csv("Data/data.csv")

In [9]:
df

,hotelName,hotelAddress,hotelDescription,pricePerNight,scoreGeneral,scoreValueForMoney,scoreLocation,scoreClean
0,Melio Hotel Eilat,"\nSderot Hativat HaNegev 20, Eilat\n","\nSet in Eilat, within 1.2 km of Kisuski Beach...",\n₪ 286\n,"7.9 Good · 1,075 reviews",8.4,7.8,8.4
1,Badolina Ein Gedi Glamping,"\nEin Gedi Street, Ein Gedi\n","\nDead Sea Camping is set in Ein Gedi, within ...",\n₪ 130\n,8.5 Very good · 814 reviews,7.7,9.2,8.3
2,Daniel Herzliya Hotel,"\n60 Ramat Yam Street, Herzelia \n","\nSet right on the beach, in the heart of Herz...",\n₪ 777\n,7.9 Good · 162 reviews,7.5,9.0,8.0
3,Merom Golan Resort,"\nKibbutz Merom Golan, Merom Golan\n",\nMerom Golan Resort offers quiet mountain acc...,\n₪ 983\n,"8.7 Fabulous · 1,400 reviews",7.9,9.4,8.9
4,Leonardo Club Hotel Dead Sea - All Inclusive,"\nEin Bokek (Dead sea), Ein Bokek\n",\nThis all-inclusive deluxe hotel is set a pri...,"\n₪ 1,308\n","7.7 Good · 6,414 reviews",7.4,8.7,7.5
...,...,...,...,...,...,...,...,...
6076,צימר נוף העמק,"\n28 נוף העמק, Nof Kinneret\n","\nSituated in Nof Kinneret, 4.7 km from Israel...","\n₪ 1,454\n",8.0 Very good · 1 review,10.0,10.0,10.0
6077,Supreme 4BR Villa With Garden and Sea View,"\n55 Jabotinski Street, Nahariyya\n",\nSituated in Nahariyya and only 400 m from Ha...,"\n₪ 4,087\n",NaN,NaN,NaN,NaN
6078,Blue lake luxury villa for families,"\nעין אל אסד, ‘Ein el Asad\n",\nLocated in ‘Ein el Asad and only 37 km from ...,"\n₪ 4,160\n",6.0 Pleasant · 1 review,NaN,NaN,NaN
6079,Paradise by the sea,"\nדוגית, Nahariyya\n","\nParadise by the sea is located in Nahariyya,...","\n₪ 2,880\n",NaN,NaN,NaN,NaN


In [8]:
df.shape

(6081, 8)

Currently the data size is 6081 * 8 = 48,648.

We're not done getting all the data yet, and we'll have a few more columns, like distanceToTlv and address details. These columns will be added during and after the Data Handling step.

Finally the data size will be XXXX.

## Step 3: Data Handling

At this point, we will need to handle our data and organize it. For example, in the hotel data we crawled, some hotels do not have ratings on Booking.com. In addition, we will have to deal with duplicate hotels and outliers that do not reflect most of our data. Let's go for it!

### Auxiliary Functions

In [ ]:
#extract the adress to three new columns
#for check


# Create a sample dataframe
data = {'Full Address': ['geulim, Holon', '25 HaYarkon, Tel Aviv', '10 HaYarkon, Jerusalem']}
df1 = pd.DataFrame(data)

# Create 3 new columns
df1['House Number'] = ''
df1['Street'] = ''
df1['City'] = ''

# Extract the data
df1['House Number'] = df1['Full Address'].str.split(' ').str[0]
df1['Street'] = df1['Full Address'].str.extract(r'\s(.*?)\,')
df1['City'] = df1['Full Address'].str.split(',', 1).str[1].str.strip()

print(df1)

In [ ]:
#extract the adress to three new columns
#as function
def addThreeCol(df):

    #create 3 new columns
    df['House Number'] = ''
    df['Street'] = ''
    df['City'] = ''

    #extract the data
    df['House Number'] = df['Full Address'].str.split(' ').str[0]
    df['Street'] = df['Full Address'].str.extract(r'\s(.*?)\,')
    df['City'] = df['Full Address'].str.split(',', 1).str[1].str.strip()

    print(df)


In [ ]:
# add a column with the distance from TLV
 #for check
df1['distance-TLV'] = ''

city_name = df1['City'].tolist()
print(city_name)
TLVDis= []
for city in city_name:
    time.sleep(2)
    city_id = getCityID(city)
    time.sleep(1)
    city_distance = getCityDistance(city_id)
    TLVDis.append(city_distance)


for j in range(len(TLVDis)):
    df1.at[j,'distance-TLV'] = TLVDis[j]


print(df1)

In [ ]:
# add a column with the distance from TLV
 # as real function
 def addDisTlvCol(df):

    df['distance-TLV'] = ''

    city_name = df['City'].tolist()
    print(city_name)
    TLVDis= []
    
    for city in city_name:
        time.sleep(2)
        city_id = getCityID(city)
        time.sleep(1)
        city_distance = getCityDistance(city_id)
        TLVDis.append(city_distance)

    for j in range(len(TLVDis)):
        df.at[j,'distance-TLV'] = TLVDis[j]

    print(df)

In [ ]:
# Handling "/n" signs and others

def cleanSigns(df, cols):

    newDf = df.copy()

    for column in cols:
        newDf[column] = newDf[column].str.replace("[\n\t\r₪]", "", regex=True)

    return newDf

In [ ]:
# Extracting "generalScore" from a columm:

def extractNumbers(df, column):

    newDf = df.copy()
    newDf[column] = newDf[column].str.split().str[0]

    return newDf

In [ ]:
# Converting text columns to numbers

def convertToNum(df, cols):
    
    newDf = df.copy()

    for column in cols:
        newDf[column] = pd.to_numeric(df[column], errors="coerce")
    
    return newDf

#### Handling missing data:


In [ ]:
# clean all the rows with more than 3 nan (trust on that we have 11 columns)
def cleanTreshRows(df, num_max_missing_cols):
    return df.dropna(thresh = df.shape[1] - num_max_missing_cols).copy()

cleanTreshRows(df1, 1)

In [ ]:
#replace a NAN data with key word, according to the df type. 
# col_to_def_val_dict==list of all the columns

def replace_missing_values(df, col_to_def_val_dict):
    updated_df = df.fillna(col_to_def_val_dict)
    for col in updated_df.columns: 
        if updated_df[col].dtype in ['float64', 'int64']:
            updated_df[col].fillna(updated_df[col].median(), inplace = True) #replace with hezion
        if updated_df[col].dtype == 'object':
            #updated_df[col].fillna('bla', inplace = True) #replace string with bla
            updated_df[col].fillna(updated_df[col].mode()[0], inplace = True) # replaced with most frequant value

    return updated_df
col_to_def_val = {'Street':'bla', 'City':'unknown'}
replace_missing_values(df1, col_to_def_val)

#### Handling duplication:

In [ ]:
# Handling  duplication:

def count_duplicatives(df, col_name=None):
    if col_name is None:
        print("num of Duplicates",df.duplicated().sum())
        return df.duplicated().sum()
    print("num of Duplicates",df.duplicated().sum())
    return df.duplicated([col_name]).sum()

In [ ]:
#remove duplicate rows
def remove_duplicatives(df, col_name=None):
    if col_name is None:
        return df[~df.duplicated()]
    return df[~df.duplicated([col_name])]

#### Handling outliers:

In [ ]:
#find all the outliers

def findOutliears(df):
    # Specify the numeric columns
    numeric_columns = ['pricePerNight', 'scoreGeneral', 'scoreValueForMoney','scoreLocation','scoreClean']
			
    # Iterate over each numeric column
    for column in numeric_columns:
        # Calculate the z-scores for each value in the numeric column
        z_scores = np.abs(stats.zscore(df[column]))
        
        # Define the threshold for outliers
        threshold = 3
        
        # Identify the rows that contain outlier values
        outlier_rows = df[z_scores > threshold]
        
        # Print the outlier rows
        print(f"Outliers in column '{column}':")
        print(outlier_rows)
        print("\n")


findOutliears(df)

### Main Function

In [ ]:
# Main



## Back to Step 2: Data Acquisition

### Data Acquisition by API

a little description and notes about this step...

#### Auxiliary Functions

In these two following function we calculate the distance between Tel-Aviv to another city.
The calculation using an API which called GEODB in order to get the city ID and to find the distance.

In [ ]:


def getCityID(cityName):

    # Set up the API endpoint URL
    url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities"

    # Set your API key and headers
    api_key = "b357d38c99mshac61197df8fd7c2p1d5cd7jsn2bfe93de690c"
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
    }

    # Set the query parameters for the country code and name
    params = {
        "countryIds": "IL",
        "namePrefix": cityName
    }

    # Send GET request to the API
    response = requests.get(url, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()

        # Check if any cities were found
        if data["data"]:
            # Get the city ID from the first result
            city_id = data["data"][0]["id"]
            print("City ID:", city_id)
            return city_id
        else:
            print("No matching cities found.")
            return np.nan
    else:
        print("Error:", response.status_code)
        return np.nan

In [ ]:


def getCityDistance(cityID): 
    url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/54067/distance"

    querystring = {"fromCityId":"54067","distanceUnit":"km","toCityId":cityID}

    headers = {
        "X-RapidAPI-Key": "b357d38c99mshac61197df8fd7c2p1d5cd7jsn2bfe93de690c",
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
    data = response.json()

    if "data" in data:
        return data["data"]
    else:
        return None

#### Main Function

In [ ]:
###NOT important I have below the function which using the api to add new col with the distance to tlv
# cityName = "Haifa"
# city_id = getCityID(cityName)
# time.sleep(1)
# city_distance = getCityDistance(city_id)
# if city_distance is not None:
#     print("Distance:", city_distance)
# else:
#     print("Distance information not available.")


## Step 4: EDA

a little description and notes about this step...

#### Auxiliary Functions

In [ ]:
# Code:



#### Main Function

In [ ]:
# Main



write a bottom line about the corelation between distance and price...

## Step 5: Machine Learning

a little description and notes about this step...

#### Auxiliary Functions

In [ ]:
# Code:



#### Main Function

In [1]:
# Main



## Summary

a little paragraph that sums up the job

can't trust 100% third side - like the API, booking.com ...

repeat the conclusion about the corlation between the price and the distance, explain it is possible to predict ...

We enjoyed ... Recommend others to deal with Data Science...